<a href="https://colab.research.google.com/github/ThousandAI/Application-of-AI/blob/main/class02/class02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Autograd

In [1]:
import torch
import torch.nn as nn

In [2]:
# grad = True
tensor = torch.rand(3, requires_grad=True)
print(tensor)
y = tensor + 6
print(y)
z = 3*y**2 + 2
print(z)
z = z.mean()
print(z)
z.backward() # dz/d(tensor)
print(tensor.grad) # 2*tensor + 12

tensor([0.8232, 0.1471, 0.8626], requires_grad=True)
tensor([6.8232, 6.1471, 6.8626], grad_fn=<AddBackward0>)
tensor([141.6680, 115.3589, 143.2871], grad_fn=<AddBackward0>)
tensor(133.4380, grad_fn=<MeanBackward0>)
tensor([13.6464, 12.2941, 13.7253])


In [3]:
# grad = False
tensor = torch.rand(3, requires_grad=False)
print(tensor)

y = tensor + 6
print(y)
z = 3*y**2 + 2
print(z)
z = z.mean()
print(z)
z.backward() # dz/d(tensor)

tensor([0.1337, 0.1228, 0.4778])
tensor([6.1337, 6.1228, 6.4778])
tensor([114.8653, 114.4644, 127.8841])
tensor(119.0713)


RuntimeError: ignored

In [4]:
# not scalar
tensor = torch.tensor([0.37, 0.58, 0.33], requires_grad=True)
print(tensor)

y = tensor + 6
print(y)
z = 3*y**2 + 2
print(z)
z.backward()

tensor([0.3700, 0.5800, 0.3300], requires_grad=True)
tensor([6.3700, 6.5800, 6.3300], grad_fn=<AddBackward0>)
tensor([123.7307, 131.8892, 122.2067], grad_fn=<AddBackward0>)


RuntimeError: ignored

In [5]:
# grad 
tensor = torch.rand(3, requires_grad = True)
print(tensor)
y = tensor.detach()
print(y)

tensor([0.7110, 0.0037, 0.2208], requires_grad=True)
tensor([0.7110, 0.0037, 0.2208])


In [6]:
# grad 
with torch.no_grad():
  y = tensor + 2
  print(y)

tensor([2.7110, 2.0037, 2.2208])


In [7]:
# accumulated gradient
weights = torch.tensor([2., 3., 5., 7.], requires_grad=True)

for epoch in range(5):
  outputs = (3*weights).sum()
  outputs.backward()

  print(weights.grad)

tensor([3., 3., 3., 3.])
tensor([6., 6., 6., 6.])
tensor([9., 9., 9., 9.])
tensor([12., 12., 12., 12.])
tensor([15., 15., 15., 15.])


In [8]:
# empty gradient
weights = torch.tensor([2., 3., 5., 7.], requires_grad=True)

for epoch in range(5):
  outputs = (3*weights).sum()
  outputs.backward()

  print(weights.grad)

  weights.grad.zero_()

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])


## toy example

In [9]:
# toy example
x = torch.tensor([[1,-1], [2,3], [5,2]], dtype=torch.float32) # 3x2
y = torch.tensor([[1],[0],[1]], dtype=torch.float32)

w1 = torch.rand(2,3, requires_grad=True)
w2 = torch.rand(3,3, requires_grad=True)
w3 = torch.rand(3,2, requires_grad=True)
w4 = torch.rand(2,1, requires_grad=True)
relu = nn.ReLU()
sigmoid = nn.Sigmoid()
bce = nn.BCELoss()

def forward(inputs):
    inputs = torch.matmul(inputs, w1)
    inputs = relu(inputs)
    inputs = torch.matmul(inputs, w2)
    inputs = relu(inputs)
    inputs = torch.matmul(inputs, w3)
    inputs = relu(inputs)
    inputs = torch.matmul(inputs, w4)
    outputs = sigmoid(inputs)
    return outputs

# loss
def loss(y_true, y_pred):
    return bce(y_pred, y_true)


learning_rate = 0.01
epochs = 10

for epoch in range(epochs):
    # forward pass
    y_hat = forward(inputs=x)

    # loss
    bce_loss = loss(y_true=y, y_pred=y_hat)

    # backward loss
    bce_loss.backward()

    # update weights
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        w3 -= learning_rate * w3.grad
        w4 -= learning_rate * w4.grad

    # zero gradients
    w1.grad.zero_()
    w2.grad.zero_()
    w3.grad.zero_()
    w4.grad.zero_()

    if epoch % 5 == 0:
        print(f"epoch {epoch + 1}: \nw1 = {w1}\n w2 = {w2}\n w3 = {w3}\n w4 = {w4}, loss = {bce_loss:.8f}")

epoch 1: 
w1 = tensor([[0.4673, 0.1780, 0.5647],
        [0.9854, 0.0098, 0.3119]], requires_grad=True)
 w2 = tensor([[0.1375, 0.9906, 0.5325],
        [0.4080, 0.1510, 0.0965],
        [0.2548, 0.2647, 0.5390]], requires_grad=True)
 w3 = tensor([[0.5565, 0.4046],
        [0.0249, 0.5972],
        [0.3447, 0.7015]], requires_grad=True)
 w4 = tensor([[0.6718],
        [0.2977]], requires_grad=True), loss = 1.25990689
epoch 6: 
w1 = tensor([[ 0.4552,  0.1728,  0.5570],
        [ 0.9659, -0.0037,  0.2919]], requires_grad=True)
 w2 = tensor([[0.1106, 0.9813, 0.5094],
        [0.4062, 0.1504, 0.0949],
        [0.2419, 0.2603, 0.5279]], requires_grad=True)
 w3 = tensor([[ 0.5462,  0.4004],
        [-0.0173,  0.5802],
        [ 0.3151,  0.6895]], requires_grad=True)
 w4 = tensor([[0.6478],
        [0.2226]], requires_grad=True), loss = 0.99273992


# Dataset & DataLoader

In [10]:
from torch.utils.data import Dataset, DataLoader

In [11]:
class ScoreDataset(Dataset):
  def __init__(self):
    self.x = torch.tensor([[1,2,3],[4,5,6],[7,8,9],[10,11,12],[13,14,15],[16,17,18]]) # 6x3
    self.y = torch.tensor([[1],[0],[1],[1],[1],[0]]) # 6x1

  def __getitem__(self,index):
    return self.x[index], self.y[index]

  def __len__(self):
    return self.x.size(0)

In [12]:
score_dataset = ScoreDataset()

In [13]:
print(score_dataset.x)
print(score_dataset.y)

tensor([[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12],
        [13, 14, 15],
        [16, 17, 18]])
tensor([[1],
        [0],
        [1],
        [1],
        [1],
        [0]])


In [14]:
dataLoader = DataLoader(dataset=score_dataset,batch_size=2,shuffle=True)

In [15]:
for i, (x,y) in enumerate(dataLoader):
  print(f"{i}: x={x}, y={y}")

0: x=tensor([[ 7,  8,  9],
        [10, 11, 12]]), y=tensor([[1],
        [1]])
1: x=tensor([[4, 5, 6],
        [1, 2, 3]]), y=tensor([[0],
        [1]])
2: x=tensor([[13, 14, 15],
        [16, 17, 18]]), y=tensor([[1],
        [0]])
